In [1]:
required_packages <- c("tidyverse")
new_packages <- required_packages[!(required_packages %in% installed.packages()[,"Package"])]
if(length(new_packages)) install.packages(new_packages)

In [2]:
library(dplyr)
library(readr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# County-level data

In [3]:
# Import county-level data for 2007 and 2018
counties_2007 <- read_csv("data/Counties_Specific_Chronic_Conditions_2007_data.csv")
counties_2018 <- read_csv("data/Counties_Specific_Chronic_Conditions_2018_data.csv")

# Use a regular expression to filter out counties that are "Unknown"
counties_2007_filtered <- subset(counties_2007, !grepl(".* : Unknown", Bene_Geo_Desc))
counties_2018_filtered <- subset(counties_2018, !grepl(".* : Unknown", Bene_Geo_Desc))

# Use a regular expression to filter for counties that are "Unknown"
counties_2007_unknown <- subset(counties_2007, grepl(".* : Unknown", Bene_Geo_Desc))
counties_2018_unknown <- subset(counties_2018, grepl(".* : Unknown", Bene_Geo_Desc))

# Filter for counties with null values for prevalence (2007)
counties_2007_na <- counties_2007_filtered[ , c("Bene_Geo_Desc", "Bene_Geo_Cd", "Prvlnc")]
names(counties_2007_na) <- c("County", "fips", "Prevalence")
counties_2007_na <- subset(counties_2007_na, is.na(Prevalence))

# Preview data
head(counties_2007_na)

# Filter for counties with/without null values for prevalence (2018)
counties_2018_na <- counties_2018_filtered[ , c("Bene_Geo_Desc", "Bene_Geo_Cd", "Prvlnc")]
names(counties_2018_na) <- c("County", "fips", "Prevalence")
counties_2018_na <- subset(counties_2018_na, is.na(Prevalence))

counties_2018_not_na <- counties_2018_filtered[ , c("Bene_Geo_Desc", "Bene_Geo_Cd", "Prvlnc")]
names(counties_2018_not_na) <- c("County", "fips", "Prevalence")
counties_2018_not_na <- subset(counties_2018_not_na, !is.na(Prevalence))

# Preview data
head(counties_2018_na)
head(counties_2018_not_na)

Rows: 3198 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): Bene_Geo_Lvl, Bene_Geo_Desc, Bene_Geo_Cd, Bene_Age_Lvl, Bene_Demo_L...
dbl (1): Prvlnc

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 3198 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): Bene_Geo_Lvl, Bene_Geo_Desc, Bene_Geo_Cd, Bene_Age_Lvl, Bene_Demo_L...
dbl (1): Prvlnc

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


County,fips,Prevalence
<chr>,<chr>,<dbl>
Alabama : Baldwin,01003,NA
Alabama : Bibb,01007,NA
Alabama : Blount,01009,NA
Alabama : Bullock,01011,NA
Alabama : Butler,01013,NA
Alabama : Calhoun,01015,NA


County,fips,Prevalence
<chr>,<chr>,<dbl>
Alabama : Autauga,01001,NA
Alabama : Barbour,01005,NA
Alabama : Bibb,01007,NA
Alabama : Blount,01009,NA
Alabama : Butler,01013,NA
Alabama : Chambers,01017,NA


County,fips,Prevalence
<chr>,<chr>,<dbl>
Alabama : Baldwin,01003,0.0010
Alabama : Bullock,01011,0.0000
Alabama : Calhoun,01015,0.0018
Alabama : Choctaw,01023,0.0000
Alabama : Clay,01027,0.0000
Alabama : Colbert,01033,0.0021


In [4]:
# Year: 2007

# Total number of counties
count_2007 <- nrow(counties_2007_filtered) # 3148 possible counties
sprintf("There were %d possible counties in 2007.", count_2007)

# Number of counties with prevalence data
count_not_null_2007 <- nrow(na.omit(counties_2007_filtered)) # 1501 counties reported data
sprintf("There were %d counties with prevalence data in 2007.", count_not_null_2007)

# Number of counties without prevalence data
count_null <- nrow(counties_2007_na) # No data for 1647 counties in 2007
sprintf("There were %d counties without prevalence data in 2007.", count_null)

# Number of counties with zero prevalence
count_zero <- nrow(subset(counties_2007_filtered, Prvlnc == 0)) # 1045 counties had zero prevalence
sprintf("There were %d counties with zero ASD prevalence in 2007.", count_zero)

[1] "There were 3148 possible counties in 2007."

[1] "There were 1501 counties with prevalence data in 2007."

[1] "There were 1647 counties without prevalence data in 2007."

[1] "There were 1045 counties with zero ASD prevalence in 2007."

In [5]:
# Year: 2018

# Total number of counties
count_2018 <- nrow(counties_2018_filtered) # 3148 possible counties
sprintf("There were %d possible counties in 2018.", count_2018)

# Number of counties with prevalence data
count_not_null_2018 <- nrow(na.omit(counties_2018_filtered)) # 1625 counties reported data
sprintf("There were %d counties with prevalence data in 2018.", count_not_null_2018)

# Number of counties without prevalence data
count_null <- nrow(counties_2018_na) # No data for 1523 counties in 2018
sprintf("There were %d counties without prevalence data in 2018.", count_null)

# Number of counties with zero prevalence
count_zero <- nrow(subset(counties_2018_filtered, Prvlnc == 0)) # 382 counties had zero prevalence
sprintf("There were %d counties with zero ASD prevalence in 2018.", count_zero)

[1] "There were 3148 possible counties in 2018."

[1] "There were 1625 counties with prevalence data in 2018."

[1] "There were 1523 counties without prevalence data in 2018."

[1] "There were 382 counties with zero ASD prevalence in 2018."

In [6]:
# Check whether prevalence data exists for "Unknown" counties
count_unknown_2007 <- sum(complete.cases(counties_2007_unknown)) # No data for "Unknown" counties in 2007
count_unknown_2018 <- sum(complete.cases(counties_2018_unknown)) # No data for "Unknown" counties in 2018

sprintf("There were %d 'Unknown' counties with data in 2007 and %d 'Unknown' counties with data in 2018.", count_unknown_2007, count_unknown_2018)

[1] "There were 0 'Unknown' counties with data in 2007 and 0 'Unknown' counties with data in 2018."

In [7]:
# Import 2014-2018 ACS data, which was downloaded from data.census.gov
acs5y_2018 <- read_csv("data/ACS5Y2018/ACSDP5Y2018.DP05-Data.csv", skip = 1)

# Preview data
head(acs5y_2018)

New names:
• `Estimate!!SEX AND AGE!!Total population!!18 years and over` ->
  `Estimate!!SEX AND AGE!!Total population!!18 years and over...43`
• `Margin of Error!!SEX AND AGE!!Total population!!18 years and over` ->
  `Margin of Error!!SEX AND AGE!!Total population!!18 years and over...44`
• `Estimate!!SEX AND AGE!!Total population!!65 years and over` ->
  `Estimate!!SEX AND AGE!!Total population!!65 years and over...49`
• `Margin of Error!!SEX AND AGE!!Total population!!65 years and over` ->
  `Margin of Error!!SEX AND AGE!!Total population!!65 years and over...50`
• `Estimate!!SEX AND AGE!!Total population!!18 years and over` ->
  `Estimate!!SEX AND AGE!!Total population!!18 years and over...51`
• `Margin of Error!!SEX AND AGE!!Total population!!18 years and over` ->
  `Margin of Error!!SEX AND AGE!!Total population!!18 years and over...52`
• `Estimate!!SEX AND AGE!!Total population!!65 years and over` ->
  `Estimate!!SEX AND AGE!!Total population!!65 years and over...59`
• `Margin

Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Margin of Error!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),⋯,"Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races",Percent Estimate!!Total housing units,Percent Margin of Error!!Total housing units,"Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female",...359
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,⋯,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
0500000US01001,"Autauga County, Alabama",55200,*****,26874,147,28326,147,94.9,1.0,⋯,0.5,(X),(X),41304,(X),47.8,0.3,52.2,0.3,NA
0500000US01003,"Baldwin County, Alabama",208107,*****,101188,242,106919,242,94.6,0.4,⋯,0.3,(X),(X),159193,(X),47.9,0.2,52.1,0.2,NA
0500000US01005,"Barbour County, Alabama",25782,*****,13697,71,12085,71,113.3,1.3,⋯,0.6,(X),(X),19991,(X),53.3,0.2,46.7,0.2,NA
0500000US01007,"Bibb County, Alabama",22527,*****,12152,180,10375,180,117.1,3.8,⋯,0.4,(X),(X),17672,(X),53.7,0.3,46.3,0.3,NA
0500000US01009,"Blount County, Alabama",57645,*****,28434,137,29211,137,97.3,0.9,⋯,0.3,(X),(X),42627,(X),48.6,0.3,51.4,0.3,NA
0500000US01011,"Bullock County, Alabama",10352,*****,5663,71,4689,71,120.8,3.4,⋯,0.3,(X),(X),8202,(X),55.2,0.1,44.8,0.1,NA


In [8]:
# Extract the last five characters from the Geography ID, which represents a county's FIPS code
acs5y_2018 <- acs5y_2018 %>% mutate(fips = substr(Geography, nchar(Geography) - 4, nchar(Geography)))

# Filter for and rename relevant columns
acs5y_2018 <- acs5y_2018[ , c("Geographic Area Name", "Estimate!!SEX AND AGE!!Total population", "Estimate!!SEX AND AGE!!Total population!!65 to 74 years", "Estimate!!SEX AND AGE!!Total population!!75 to 84 years", "Estimate!!SEX AND AGE!!Total population!!85 years and over", "fips")]
names(acs5y_2018) <- c("County_State", "Estimated_Population", "Estimated_65to74", "Estimated_75to84", "Estimated_85orOlder", "fips")

# Filter out Puerto Rico
acs5y_2018_filtered <- subset(acs5y_2018, !grepl(".*, Puerto Rico", County_State))

In [9]:
# Convert relevant columns to numeric data type
acs5y_2018_filtered <- acs5y_2018_filtered %>% mutate_at(c("Estimated_Population", "Estimated_65to74", "Estimated_75to84", "Estimated_85orOlder"), as.numeric)

# Sum across age groups
acs5y_2018_filtered <- acs5y_2018_filtered %>% mutate(olderAdultSum = rowSums(across(c("Estimated_65to74", "Estimated_75to84", "Estimated_85orOlder"))))

# Preview data
head(acs5y_2018_filtered)

County_State,Estimated_Population,Estimated_65to74,Estimated_75to84,Estimated_85orOlder,fips,olderAdultSum
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
"Autauga County, Alabama",55200,4711,2524,815,01001,8050
"Baldwin County, Alabama",208107,24551,12165,3949,01003,40665
"Barbour County, Alabama",25782,2820,1392,422,01005,4634
"Bibb County, Alabama",22527,2122,1112,427,01007,3661
"Blount County, Alabama",57645,6132,3235,866,01009,10233
"Bullock County, Alabama",10352,1011,430,175,01011,1616


In [10]:
# Merge counties with/without prevalence data with the Census population data based on FIPS code
counties_2018_na_population <- merge(counties_2018_na, acs5y_2018_filtered, by = "fips", all.x = TRUE)
counties_2018_not_na_population <- merge(counties_2018_not_na, acs5y_2018_filtered, by = "fips", all.x = TRUE)

# Preview data
head(counties_2018_na_population)
head(counties_2018_not_na_population)

,fips,County,Prevalence,County_State,Estimated_Population,Estimated_65to74,Estimated_75to84,Estimated_85orOlder,olderAdultSum
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,01001,Alabama : Autauga,NA,"Autauga County, Alabama",55200,4711,2524,815,8050
2,01005,Alabama : Barbour,NA,"Barbour County, Alabama",25782,2820,1392,422,4634
3,01007,Alabama : Bibb,NA,"Bibb County, Alabama",22527,2122,1112,427,3661
4,01009,Alabama : Blount,NA,"Blount County, Alabama",57645,6132,3235,866,10233
5,01013,Alabama : Butler,NA,"Butler County, Alabama",20025,2081,1168,557,3806
6,01017,Alabama : Chambers,NA,"Chambers County, Alabama",33826,3775,1932,702,6409


,fips,County,Prevalence,County_State,Estimated_Population,Estimated_65to74,Estimated_75to84,Estimated_85orOlder,olderAdultSum
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,01003,Alabama : Baldwin,0.0010,"Baldwin County, Alabama",208107,24551,12165,3949,40665
2,01011,Alabama : Bullock,0.0000,"Bullock County, Alabama",10352,1011,430,175,1616
3,01015,Alabama : Calhoun,0.0018,"Calhoun County, Alabama",115098,11489,5847,2050,19386
4,01023,Alabama : Choctaw,0.0000,"Choctaw County, Alabama",13075,1647,942,295,2884
5,01027,Alabama : Clay,0.0000,"Clay County, Alabama",13378,1544,859,249,2652
6,01033,Alabama : Colbert,0.0021,"Colbert County, Alabama",54495,6003,3283,1216,10502


In [11]:
# Calculate and display the sum of all people living in the counties without associated prevalence data and the total U.S. population in 2018
sum_population <- sum(counties_2018_na_population$Estimated_Population, na.rm = TRUE)
sum_population_formatted <- format(sum_population, big.mark = ",")
sprintf("In 2018, there were %s people living in the counties without associated prevalence data.", sum_population_formatted)

sum_total_population <- sum(acs5y_2018_filtered$Estimated_Population, na.rm = TRUE)
sum_total_population_formatted <- format(sum_total_population, big.mark = ",")
sprintf("The total population in 2018 was approximately %s.", sum_total_population_formatted)

# Calculate and display the sum of people 65 or older living in the counties without associated prevalence data and the total U.S. older adult population in 2018 
sum_65_or_older <- sum(counties_2018_na_population$olderAdultSum, na.rm = TRUE)
sum_65_or_older_formatted <- format(sum_65_or_older, big.mark = ",")
sprintf("In 2018, there were %s people 65 or older living in the counties without associated prevalence data.", sum_65_or_older_formatted)

sum_total_65_or_older <- sum(acs5y_2018_filtered$olderAdultSum, na.rm = TRUE)
sum_total_65_or_older_formatted <- format(sum_total_65_or_older, big.mark = ",")
sprintf("The total older adult population in 2018 was approximately %s.", sum_total_65_or_older_formatted)

write.csv(acs5y_2018_filtered, "data/Filtered/Population by county in 2018.csv", row.names = FALSE)

[1] "In 2018, there were 31,356,122 people living in the counties without associated prevalence data."

[1] "The total population in 2018 was approximately 322,903,030."

[1] "In 2018, there were 5,616,599 people 65 or older living in the counties without associated prevalence data."

[1] "The total older adult population in 2018 was approximately 49,238,581."